# Question Answering

Recall the overall workflow for retrieval augmented generation

![](rag.jpeg)

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`. Now we'll abord `Question Answering`.

Let's load our vectorDB. 

In [ ]:
import os
import openai

In [ ]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "what are major topics for this class?"
docs = vectordb.similarity_search(question, k=3)
len(docs)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)

## RetrievalQA Chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

## Prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Is probability a class topic?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

In [ ]:
result["source_documents"][0]

## RetrievalQA Chain Types

In [ ]:
# Map Reduce technique
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [ ]:
result = qa_chain_mr({"query": question})

In [ ]:
result["result"]

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = str(os.getenv("LANGCHAIN_TRACING_V2"))
LANGCHAIN_ENDPOINT = str(os.getenv("LANGCHAIN_ENDPOINT"))
LANGCHAIN_API_KEY = str(os.getenv("LANGCHAIN_API_KEY"))


In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

result = qa_chain_mr({"query": question})
result["result"]

In [ ]:
result = qa_chain_mr({"query": question})

In [ ]:
result["result"]